# oneM2M - Access Control

This notebook demonstrates how access control to resources can be done in oneM2M.

- Create an &lt;ACP> resource with different credentials for a new originator
- Create a second &lt;AE> resource with the new access controls policy
- Succeed to add a &lt;Container> to the second &lg;AE> resource
- Fail to update the second &lt;AE> resource

## Intitialization
The section does import necessary modules and configurations.

In [ ]:
from init import *

## Create an &lt;ACP> Resource

Access Control Policies are used to associate access control with credentials. They define the rules to for access control to resources. Each &lt;ACP> resource has two sections:

- **pv (Privileges)** : The actual privileges defined by this policy.
- **pvs (Self-Privileges)** : This defines the privileges necessary to access and control the &lt;ACP> resource itself.

Each section has at least the following two parameters:

- **acor (accessControlOriginators)** : This list includes the Originator information. The parameter comprises a list of domain, CSE-IDs, AE-IDs, the resource-ID of a &lt;Group> resource that contains &lt;AE> or &lt;remoteCSE> as member or Role-ID.
- **acop (accessControlOperations)** : This number represents a bit-field of privileges. The following table shows the mapping:

| Value | Interpretation |
|-------|----------------|
| 1     | CREATE         |
| 2     | RETRIEVE       |
| 4     | UPDATE         |
| 8     | DELETE         |
| 16    | NOTIFY         |
| 32    | DISCOVERY      |

The following request creates a new &lt;ACP> that allows the originator *abc:xyz* only to send CREATE, RETRIEVE, NOTIFY and DISCOVERY requests to resources that have this &lt;ACP> resource assigned.

In [ ]:
POST (
    url,
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=1'   # Content is JSON, and represents an <ACP> resource
    },
    
    # Request Body
    '''
    {
      "m2m:acp": {
        "rn":"myACP",
        "pv": {
            "acr": {
                "acor": [ "abc:xyz" ],
                "acop": 51
            }
        },
        "pvs": {
            "acr": {
                "acor": [ "admin:admin" ],
                "acop": 63
            }
        }
      }
    }
    '''
)



## Create a second &lt;AE> Resource with the new &lt;ACP>

We now create a new &lt;AE> resource that uses the just created &lt;ACP>.

**IMPORTANT** : Please copy from the previous request's result body above the **ri** field's value (the resource identifier). Replace the value of the *acpi* attribute in the request below.



In [ ]:
POST (
    url,
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
    },
    
    # Request Body
    '''
    {   
      "m2m:ae": {
        "rn": "secondAE",
        "api": "AE",
        "acpi" : [ "/in-cse/acp-180113392" ],
        "rr": true
      }
    }
    '''
)

## Try to Create &lt;Container> under the second &lt;AE> Resource

We will update a &lt;Container> resource under the second &lt;AE> resource with the originator of *abc:xyz*. This should work since this originator is allowed to send CREATE requests.

In [ ]:
 POST (
    url + '/secondAE',
    
    # Request Headers
    {
        'X-M2M-Origin' : "abc:xyz",                # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cnt": {
        "rn":"secondContainer"
      }
    }
    '''
)

## Try to Update the second &lt;AE> Resource

Now we try to update the new &lt;AE> resource (add a *lbl* attribute) with the other originator, *abc:xyz*. This should fail, since the associated &lt;ACP> doesn't allow UPDATE requests.

In [ ]:
PUT (
    url + '/secondAE',
    
    # Request Headers
    {
        'X-M2M-Origin' : 'abc:xyz',                # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
    },
    
    # Request Body
    '''
    {
      "m2m:ae": {
        "lbl": [ "test/test" ]
      }
    }
    '''
)